In [2]:
import pennylane as qml
import numpy as np
import tensorflow as tf

2021-12-22 20:36:07.676574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-22 20:36:07.676623: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
dev = qml.device('default.qubit', wires=4)

In [4]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)

In [5]:
def generator(w, **kwargs):
    qml.Hadamard(wires=1)
    qml.RX(w[0], wires=1)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=1)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=1)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[1, 2])
    qml.RX(w[6], wires=1)
    qml.RY(w[7], wires=1)
    qml.RZ(w[8], wires=1)

In [6]:
def swap_test():
    qml.Hadamard(wires=3)
    qml.CSWAP(wires=[3, 0, 1])
    qml.Hadamard(wires=3)

In [7]:
@qml.qnode(dev, interface="tf")
def overlap(phi, theta, omega, gen_weights):
    real([phi, theta, omega])
    generator(gen_weights)
    swap_test()
    return qml.expval(qml.PauliZ(3))

2021-12-22 20:36:13.227339: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-22 20:36:13.227559: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-22 20:36:13.227617: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-SWX): /proc/driver/nvidia/version does not exist
2021-12-22 20:36:13.229601: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))

gen_weights = tf.Variable(init_gen_weights)

In [9]:
opt = tf.keras.optimizers.SGD(0.4)

In [10]:
cost = lambda: 1 - overlap(phi, theta, omega, gen_weights)
for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = 0.4184112029329162
Step 5: cost = 0.11036778096660871
Step 10: cost = 0.01880582736704839
Step 15: cost = 0.0035795566815230995
Step 20: cost = 0.0007667133184362518
Step 25: cost = 0.00017145415360786398
Step 30: cost = 3.8852997832572456e-05
Step 35: cost = 8.84255200905848e-06
Step 40: cost = 2.015685248379917e-06
Step 45: cost = 4.59783669470859e-07


In [11]:
obs_real = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]
obs_gen = [qml.PauliX(1), qml.PauliY(1), qml.PauliZ(1)]

bloch_vector_real = qml.map(real, obs_real, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs_gen, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.21694187  0.45048443 -0.8660254 ]
Generator Bloch vector: [-0.21651352  0.45036017 -0.86619702]
